In [1]:
!pip install evaluate
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00


In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

2024-04-30 06:39:59.575707: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 06:39:59.575873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 06:39:59.720722: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
sum_datasets = load_dataset("hugcyp/LCSTS")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
sum_datasets

DatasetDict({
    train: Dataset({
        features: ['summary', 'text'],
        num_rows: 2400591
    })
    validation: Dataset({
        features: ['summary', 'text'],
        num_rows: 8685
    })
    test: Dataset({
        features: ['summary', 'text'],
        num_rows: 725
    })
})

In [5]:
sum_datasets = load_dataset("hugcyp/LCSTS",split="train[:10000]")

In [6]:
sum_datasets

Dataset({
    features: ['summary', 'text'],
    num_rows: 10000
})

In [7]:
sum_datasets = sum_datasets.train_test_split(test_size=0.2)
sum_datasets

DatasetDict({
    train: Dataset({
        features: ['summary', 'text'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['summary', 'text'],
        num_rows: 2000
    })
})

In [8]:
sum_datasets["train"][0]

{'summary': '新民评论：罚丁书苗25亿意义深远',
 'text': '加大对行贿者的惩治力度，应当以人身处罚和财产处罚并重，让行贿者把非法获利“吐”出来，防止“一人坐牢，全家享福”的不正常现象。对丁书苗案的判决，能否在刑法修订、行贿成本有望提高的背景之下，成为一个具有典型意义的判例，令人期待。'}

In [9]:
tokenizer = AutoTokenizer.from_pretrained("uer/t5-base-chinese-cluecorpussmall")
tokenizer

tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='uer/t5-base-chinese-cluecorpussmall', vocab_size=21229, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [10]:
def process_func(exmaples):
    contents = ["摘要生成: \n" + e for e in exmaples["text"]]
    inputs = tokenizer(contents, max_length=384, truncation=True)
    labels = tokenizer(text_target=exmaples["summary"], max_length=64, truncation=True)
    inputs["labels"] = labels["input_ids"]
    return inputs

In [11]:
tokenized_ds = sum_datasets.map(process_func, batched=True)
tokenized_ds

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['summary', 'text', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['summary', 'text', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [13]:
print(tokenized_ds["train"][0])

{'summary': '新民评论：罚丁书苗25亿意义深远', 'text': '加大对行贿者的惩治力度，应当以人身处罚和财产处罚并重，让行贿者把非法获利“吐”出来，防止“一人坐牢，全家享福”的不正常现象。对丁书苗案的判决，能否在刑法修订、行贿成本有望提高的背景之下，成为一个具有典型意义的判例，令人期待。', 'input_ids': [101, 3036, 6206, 4495, 2768, 131, 1217, 1920, 2190, 6121, 6594, 5442, 4638, 2674, 3780, 1213, 2428, 8024, 2418, 2496, 809, 782, 6716, 1905, 5385, 1469, 6568, 772, 1905, 5385, 2400, 7028, 8024, 6375, 6121, 6594, 5442, 2828, 7478, 3791, 5815, 1164, 100, 1402, 100, 1139, 3341, 8024, 7344, 3632, 100, 671, 782, 1777, 4286, 8024, 1059, 2157, 775, 4886, 100, 4638, 679, 3633, 2382, 4385, 6496, 511, 2190, 672, 741, 5728, 3428, 4638, 1161, 1104, 8024, 5543, 1415, 1762, 1152, 3791, 934, 6370, 510, 6121, 6594, 2768, 3315, 3300, 3307, 2990, 7770, 4638, 5520, 3250, 722, 678, 8024, 2768, 711, 671, 702, 1072, 3300, 1073, 1798, 2692, 721, 4638, 1161, 891, 8024, 808, 782, 3309, 2521, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [12]:
tokenizer.decode(tokenized_ds["train"][0]["input_ids"])

'[CLS] 摘 要 生 成 : 银 行 将 逐 渐 停 发 磁 条 卡 ， 但 磁 条 卡 中 包 含 重 要 账 户 信 息 ， 淘 汰 后 要 谨 慎 处 理 ， 防 止 被 恶 意 使 用 。 一 些 不 法 分 子 甚 至 出 价 上 百 元 收 购 淘 汰 卡 。 银 行 卡 落 到 不 法 分 子 手 中 ， 就 可 能 被 用 来 逃 税 、 避 债 和 诈 骗 ！ 若 卡 被 用 作 违 法 犯 罪 ， 原 卡 主 可 能 受 牵 连 。 [SEP]'

In [13]:
tokenizer.decode(tokenized_ds["train"][0]["labels"])

'[CLS] 银 行 提 醒 ： 磁 条 银 行 卡 淘 汰 后 别 乱 扔 ！ [SEP]'

In [14]:
model = AutoModelForSeq2SeqLM.from_pretrained("uer/t5-base-chinese-cluecorpussmall")


pytorch_model.bin:   0%|          | 0.00/858M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [15]:
!pip install rouge_chinese

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
import numpy as np
from rouge_chinese import Rouge

rouge = Rouge()

def compute_metric(evalPred):
    predictions, labels = evalPred
    decode_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decode_preds = [" ".join(p) for p in decode_preds]
    decode_labels = [" ".join(l) for l in decode_labels]
    scores = rouge.get_scores(decode_preds, decode_labels, avg=True)
    return {
        "rouge-1": scores["rouge-1"]["f"],
        "rouge-2": scores["rouge-2"]["f"],
        "rouge-l": scores["rouge-l"]["f"],
    }

In [20]:
args = Seq2SeqTrainingArguments(
    output_dir="./summary",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    logging_steps=8,
    evaluation_strategy="steps",
    eval_steps=200,
    save_strategy="epoch",
    metric_for_best_model="rouge-l",
    predict_with_generate=True,
    report_to=['tensorboard']
)

In [21]:
trainer = Seq2SeqTrainer(
    args=args,
    model=model,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    compute_metrics=compute_metric,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer)
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [22]:
trainer.train()

Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
200,7.464200,7.074858,0.011266,0.000236,0.010595
400,7.000800,6.643053,0.083521,0.024422,0.073969
600,6.830500,6.479981,0.089810,0.027472,0.079117


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=750, training_loss=7.258201690673828, metrics={'train_runtime': 948.8402, 'train_samples_per_second': 25.294, 'train_steps_per_second': 0.79, 'total_flos': 3313311337635840.0, 'train_loss': 7.258201690673828, 'epoch': 3.0})

In [26]:
from transformers import pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0)

In [32]:
pipe("摘要生成:\n" + sum_datasets["test"][-1]["text"], max_length=64, do_sample=True)

[{'generated_text': '份 学 生 上 中 中 年 开 了 一 不 一 上 生 自 会 将 会 、 自 的 的 来 中 ， 发 ： 中 主 了 定 实 。 后 大 有 一 的 人'}]

In [33]:
sum_datasets["test"][-1]["summary"]

'“头”等大事枕边收纳'

## new model

In [38]:
tokenizer = AutoTokenizer.from_pretrained("lemon234071/t5-base-Chinese")


tokenizer_config.json:   0%|          | 0.00/258 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/716k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [39]:
tokenized_ds = sum_datasets.map(process_func, batched=True)
tokenized_ds

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['summary', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['summary', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [40]:
model = AutoModelForSeq2SeqLM.from_pretrained("lemon234071/t5-base-Chinese")


pytorch_model.bin:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

In [41]:
args = Seq2SeqTrainingArguments(
    output_dir="./summary",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    logging_steps=8,
    evaluation_strategy="steps",
    eval_steps=200,
    save_strategy="epoch",
    metric_for_best_model="rouge-l",
    predict_with_generate=True,
    report_to=['tensorboard']
)

In [42]:
trainer = Seq2SeqTrainer(
    args=args,
    model=model,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    compute_metrics=compute_metric,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer)
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [43]:
trainer.train()

Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
200,4.482700,3.660512,0.319598,0.193150,0.286869
400,4.267500,3.429786,0.337298,0.206873,0.300547
600,4.104200,3.366799,0.343168,0.210062,0.304882


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=750, training_loss=4.802234703063965, metrics={'train_runtime': 979.7099, 'train_samples_per_second': 24.497, 'train_steps_per_second': 0.766, 'total_flos': 2839954114953216.0, 'train_loss': 4.802234703063965, 'epoch': 3.0})

In [44]:
from transformers import pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0)

In [69]:
pipe("摘要生成:\n" + sum_datasets["test"][10]["text"], max_length=64, do_sample=True)

[{'generated_text': '<extra_id_0>海口首条水上巴士航线试运行'}]

In [64]:
sum_datasets["test"][10]["summary"]

'海口首条水上巴士将启航航线从万绿园至西秀海滩'

In [62]:
sum_datasets["test"][10]

{'summary': '海口首条水上巴士将启航航线从万绿园至西秀海滩',
 'text': '据了解，海口首条水上巴士航线计划9月中旬试运行，届时市民可以乘坐水上巴士从万绿园直达西秀海滩。线路实行分段收费，初步票价单程全程30元，往返全程60元。届时将会投放一艘88座的船舶用于该航线的运行'}